In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork, Trajectory
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch_geometric.transforms as T

from models import GTNModel, GTCModel, GAEModel, Node2VecModel, GCNEncoder, Traj2VecModel
from evaluation.tasks import TravelTimeEstimation, NextLocationPrediciton

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
network = RoadNetwork()
network.load("../../osm_data/porto")
trajectory = Trajectory("../../datasets/trajectories/Porto/road_segment_map_final.csv", nrows=10000).generate_TTE_datatset()

traj_features = pd.read_csv("../../datasets/trajectories/Porto/speed_features_unnormalized.csv")
traj_features.set_index(["u", "v", "key"], inplace=True)
traj_features["util"] = (traj_features["util"] - traj_features["util"].min()) / (traj_features["util"].max() - traj_features["util"].min())  # min max normalization
traj_features["avg_speed"] = (traj_features["avg_speed"] - traj_features["avg_speed"].min()) / (traj_features["avg_speed"].max() - traj_features["avg_speed"].min())  # min max normalization
traj_features.fillna(0, inplace=True)

# data = network.generate_road_segment_pyg_dataset(drop_labels=["highway_enc"])
data_roadclf = network.generate_road_segment_pyg_dataset(include_coords=True, drop_labels=["highway_enc"], traj_data=None)
data_meanspeed = network.generate_road_segment_pyg_dataset(include_coords=True, drop_labels=["avg_speed"], traj_data=traj_features.copy())
data_rest = network.generate_road_segment_pyg_dataset(include_coords=True, traj_data=None)

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

In [4]:
adj_bi = np.loadtxt("./gtn_precalc_adj/traj_adj_k_2.gz") # for traj2vec 'traj_adj_k_1_False_no_selfloops_smoothed'
adj_bi_3 = np.loadtxt("./gtn_precalc_adj/traj_adj_k_3.gz")
# adj_for = np.loadtxt("./gtn_precalc_adj/traj_adj_k_1_False.gz")

In [ ]:
print(adj[108, 130:140])

In [ ]:
walks = Traj2Vec.traj_walk(adj, 5, 10000*[0], 10)
print(walks)

In [ ]:
from _walker import random_walks as _random_walks
from scipy import sparse

A = sparse.csr_matrix(adj)
indptr = A.indptr.astype(np.uint32)
indices = A.indices.astype(np.uint32)
weights = A.data.astype(np.float32)

_random_walks(indptr, indices, weights, [100,100,100], 5, 6)

In [ ]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
traj2vec = Traj2VecModel(
            data,
            network,
            adj,
            device=device,
            emb_dim=128,
            walk_length=30,
            context_size=5,
            walks_per_node=25,
            num_neg=10,
        )
traj2vec.train(epochs=200)

In [ ]:
torch.save(traj2vec.state_dict(), "modelt.pt")

In [ ]:
data.x = None
data = T.OneHotDegree(128)(data)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# precalc adj matrices
GTCModel(data_rest, device, network, trajectory, k=6, bidirectional=False, add_self_loops=True)

In [5]:
models = []
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
# model = GTCModel(data_rest, device, network, adj=adj_bi)
# model2 = GTCModel(data_roadclf, device, network, trajectory, adj=adj_for)
# model2 = GTNModel(data2, device, network, trajectory, load_traj_adj_path="./gtn_precalc_adj/traj_adj_k_1.gz", norm=True)
# model3 = GAEModel(data2, device=device, encoder=GCNEncoder, emb_dim=128, layers=1)
# model4 = GAEModel(data2, device=device, encoder=GCNEncoder, emb_dim=128, layers=1)
# model5 = Node2VecModel(data_roadclf, device=device, q=4, p=1)
#model6 = Traj2VecModel(data_roadclf, network, adj, device=device, emb_dim=128, walk_length=30, context_size=5, walks_per_node=25, num_neg=10)

# models.extend([(model, 5000), (model2, 5000)]) # (model3, 5000), (model4, 5000)

In [20]:
model.train_data.x.shape

torch.Size([11331, 33])

In [7]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# for k in [1]:
#     model = GTNModel(data, device, network, trajectory, load_traj_adj_path="./traj_adj_k_{}.gz".format(k))
#     model.train(epochs=1000)
#     models.append(model)

model.train(epochs=20000)

Epoch: 1000, avg_loss: 1.034002430677414
Epoch: 2000, avg_loss: 1.017740310192108
Epoch: 3000, avg_loss: 1.0108185515403747
Epoch: 4000, avg_loss: 1.0068407852202654
Epoch: 5000, avg_loss: 1.004067036151886
Epoch: 6000, avg_loss: 1.0019121697843074
Epoch: 7000, avg_loss: 1.0001667389529092
Epoch: 8000, avg_loss: 0.9987443603128194
Epoch: 9000, avg_loss: 0.9975816947552892
Epoch: 10000, avg_loss: 0.9966281504809856
Epoch: 11000, avg_loss: 0.9958320065086538
Epoch: 12000, avg_loss: 0.9951536188572645
Epoch: 13000, avg_loss: 0.9945690607153452
Epoch: 14000, avg_loss: 0.9940654079658644
Epoch: 15000, avg_loss: 0.9936205333948135
Epoch: 16000, avg_loss: 0.9932212658040226
Epoch: 17000, avg_loss: 0.992866878162412
Epoch: 18000, avg_loss: 0.99254868139161
Epoch: 19000, avg_loss: 0.9922587922560542


In [8]:
z = model.load_emb()
z.shape

(11331, 128)

In [ ]:
print(models)

In [9]:
model.save_model(path="../model_states/gtc/")

In [ ]:
from torch_geometric.nn.norm import LayerNorm
# load node2vec emb
model5.load_model("../model_states/node2vec/model_base.pt")
z2 = model5.load_emb()
model6.load_model("../model_states/traj2vec/model_base.pt")
z3 = model6.load_emb()

norm = LayerNorm(z3.shape[1], affine=False)
z4 = norm(torch.Tensor(z2)).detach().cpu().numpy()
z5 = norm(torch.Tensor(z3)).detach().cpu().numpy()

In [53]:
gae = GAEModel(data_rest, device=device, encoder=GCNEncoder, emb_dim=128, layers=2)
gae.load_model("../model_states/gaegcn/model_base.pt")
gae_emb = gae.load_emb()
gtc = GTCModel(data_rest, device, network, adj=adj_bi)
gtc.load_model("../model_states/gtc/model_base.pt")
# gtc2 = GTCModel(data_rest, device, network, adj=adj_bi_3)
# gtc2.load_model("../model_states/gtc/model_base_k3_20k.pt")
t2v = Traj2VecModel(data_rest, network, adj_bi, device=device)
t2v.load_model("../model_states/traj2vec/model_base.pt")
z = np.concatenate([gtc.load_emb(), t2v.load_emb()], axis=1)
# z2 = np.concatenate([gtc2.load_emb(), t2v.load_emb()], axis=1)
rand_emb = np.random.randn(*z.shape)

In [42]:
gtc = GTCModel(data_rest, device, network, adj=adj_bi)
gtc.load_model("../model_states/gtc/model_base.pt")

print(gtc.load_emb())

[[-0.03461973 -0.7446656  -0.36518294 ... -0.12940633 -0.19541745
  -0.41113168]
 [-0.06902651 -0.30286184 -0.42417783 ... -0.11377949 -0.24691841
  -0.15814777]
 [-0.10037328 -0.67797256 -0.45901316 ... -0.14502342 -0.143866
  -0.5151305 ]
 ...
 [-0.03662838  0.23067358  0.02284841 ...  0.09844947 -0.02313899
   0.28437442]
 [-0.52729726 -0.24934813  0.35677475 ... -0.5402291  -0.09656639
   0.44079298]
 [-0.52729726 -0.24934813  0.35677475 ... -0.5402291  -0.09656639
   0.44079298]]


In [54]:
print(gtc.load_emb())
print("--------")
print(t2v.load_emb())

[[-0.03461973 -0.7446656  -0.36518294 ... -0.12940633 -0.19541745
  -0.41113168]
 [-0.06902651 -0.30286184 -0.42417783 ... -0.11377949 -0.24691841
  -0.15814777]
 [-0.10037328 -0.67797256 -0.45901316 ... -0.14502342 -0.143866
  -0.5151305 ]
 ...
 [-0.03662838  0.23067358  0.02284841 ...  0.09844947 -0.02313899
   0.28437442]
 [-0.52729726 -0.24934813  0.35677475 ... -0.5402291  -0.09656639
   0.44079298]
 [-0.52729726 -0.24934813  0.35677475 ... -0.5402291  -0.09656639
   0.44079298]]
--------
[[ 0.14195527  0.71312606  0.27853405 ...  0.08516737 -0.62053263
   0.17841183]
 [ 0.00569221  0.5116822   0.06596196 ...  0.01285282 -0.41922888
   0.14586426]
 [ 0.00964291  0.76392955  0.12494162 ... -0.02320814 -0.6379421
   0.17545395]
 ...
 [ 0.00233091  0.13886695  0.01223819 ...  0.01058747 -0.02673449
  -0.24037123]
 [-0.40296662 -0.1827692   0.13607268 ...  0.56599444  0.04516564
  -0.09180278]
 [-0.44287586 -0.19512364  0.13986318 ...  0.57609206  0.10952294
  -0.09739541]]


In [12]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(trajectory, test_size=0.3, random_state=69)

In [55]:
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

# n2v = models[-1]
idxs = np.arange(len(network.line_graph.nodes))
train_idx, test_idx = model_selection.train_test_split(idxs, test_size=0.2, random_state=69)
y = np.array([network.gdf_edges.loc[n]["highway_enc"] for n in network.line_graph.nodes])

# for m, e in models:
    # m.train(epochs=e)
    # zn = m.load_emb()
    # zcn = np.concatenate((zn, z2), axis=1)
    # zct = np.concatenate((zn, z3), axis=1)
    # zcnn = np.concatenate((zn, z4), axis=1)
    # zctn = np.concatenate((zn, z5), axis=1)
    # X = z # embedding for each node
eva = [z, gtc.load_emb(), gae_emb, rand_emb]
for X in eva:
    X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]

    lm = linear_model.LogisticRegression(multi_class="multinomial", max_iter=1000)
    # lm.fit(X_train, y_train)
    scorer = make_scorer(metrics.f1_score, average="macro")
    print(np.mean(cross_val_score(estimator=lm, X=X, y=y, scoring=scorer, cv=5)))
    #print(metrics.classification_report(y_test, lm.predict(X_test)))

0.8461625599346931
0.8599389555709361
0.6101262337702813
0.05428793153941784


In [52]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.neural_network import MLPRegressor

tf = pd.read_csv("../../datasets/trajectories/Porto/speed_features_unnormalized.csv")
tf.set_index(["u", "v", "key"], inplace=True)
map_id = {j: i for i, j in enumerate(network.line_graph.nodes)}
tf["idx"] = tf.index.map(map_id)
tf.sort_values(by="idx", axis=0, inplace=True)

# idxs = np.arange(len(network.line_graph.nodes))
#train_idx, test_idx = model_selection.train_test_split(idxs, test_size=0.2, random_state=69)

y = tf["avg_speed"]
y.fillna(0, inplace=True)
y = y.round(2)
y = y.values

# for m, e in models:
#     m.train(epochs=e)
    
    # zn = m.load_emb()
    # zcn = np.concatenate((zn, z2), axis=1)
    # zct = np.concatenate((zn, z3), axis=1)
    # zcnn = np.concatenate((zn, z4), axis=1)
    # zctn = np.concatenate((zn, z5), axis=1)
    # X = z # embedding for each node
eva = [z, gtc.load_emb(), gae_emb, rand_emb] # z, z2, gtc.load_emb(), gtc2.load_emb(), gae_emb, rand_emb]
for X in eva:
    decoder = MLPRegressor(hidden_layer_sizes=(1024), random_state=88, max_iter=30)

    # X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]

    # decoder.fit(X_train, y_train)
    scorer = make_scorer(metrics.mean_absolute_error)
    print(np.mean(cross_val_score(estimator=decoder, X=X, y=y, scoring=scorer, cv=5)))

12.68308312660881
13.492779042323273
14.231388514448602
17.7632361509869


In [39]:
from evaluation.tasks.task_loader import *
from evaluation.evaluation import Evaluation
from models import ConcateAdapterModel

eva = Evaluation()
eva.register_task("meanspeed", init_meanspeed(None, network, 88))

model = ConcateAdapterModel(None, None, models=[gtc, t2v], aggregator="concate")
model2 = ConcateAdapterModel(None, None, models=[gtc2, t2v], aggregator="concate")

eva.register_model("concat", model)
eva.register_model("concat2", model2)
res = eva.run()

print(res)

[[-0.07796444 -0.55997896 -0.34868354 ...  0.08516737 -0.62053263
   0.17841183]
 [-0.17300586 -0.313582   -0.3508929  ...  0.01285282 -0.41922888
   0.14586426]
 [-0.08473617 -0.48305732 -0.47893226 ... -0.02320814 -0.6379421
   0.17545395]
 ...
 [-0.03662838  0.23067358  0.02284841 ...  0.01058747 -0.02673449
  -0.24037123]
 [-0.52729726 -0.24934813  0.35677475 ...  0.56599444  0.04516564
  -0.09180278]
 [-0.52729726 -0.24934813  0.35677475 ...  0.57609206  0.10952294
  -0.09739541]]


Current task: 100%|██████████| 1/1 [02:10<00:00, 130.97s/it]

[('meanspeed',                 MSE        MAE       RMSE
concat   366.677423  12.915396  19.113661
concat2  369.236217  12.931941  19.173809)]


In [16]:
import random
from evaluation.tasks.task_loader import *
from evaluation.evaluation import Evaluation
from models import ConcateAdapterModel

model_map = {
    "con-traj2vec-gtc": (
        ConcateAdapterModel,
        {"models": ["gtc", "traj2vec"], "aggregator": "concate"},
        "",
    ),
    "gtn": (GTNModel, {}, "../model_states/gtn/"),
    "gtc": (GTCModel, {}, "../model_states/gtc/"),
    "traj2vec": (Traj2VecModel, {}, "../model_states/traj2vec"),

}



def get_model_path_for_task(base_path, tasks, model_name):
    """This method appends the fitting model state to the path.
    The model state dict must match the dataset and task.
    For example Road Type clf task should exclude road type and therefore needs the state dict trained without road type.
    The model state dict names are normed:
        - model_base.pt for model with all features exluding speed features (except gtn, gtc) or no features (node2vec etc.)
        - model_noroad.pt for model trained with all features excluding the road label (for roadclf task)
        - model_nospeed.pt for tained with all features exluding the mean speed (for mean speed prediction task)

    Args:
        base_path (_type_): _description_
        tasks (_type_): desc
    """
    if model_name in ["node2vec", "deepwalk", "traj2vec", "toast", "pca"]:
        return os.path.join(base_path, "model_base.pt")

    # if "meanspeed" in tasks and model_name in ["gtc", "gtn"]:
    #     assert len(tasks) == 1
    #     return os.path.join(base_path, "model_nospeed.pt")
    if "roadclf" in tasks:
        assert len(tasks) == 1
        return (
            os.path.join(base_path, "model_noroad.pt")
            if model_name != "rfn"
            else os.path.join(base_path, "model_noroad.params")
        )

    return (
        os.path.join(base_path, "model_base.pt")
        if model_name != "rfn"
        else os.path.join(base_path, "model_noroad.params")
    )


def evaluate_model(args, data, network, trajectory, seed):
    """
    trains the model given by the args argument on the corresponding data

    Args:
        args (dict): Args from arg parser
        data (pyg Data): Data to train on
    """

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

    torch.cuda.set_device(1)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    transform = T.Compose(
        [
            T.ToDevice(device),
        ]
    )
    # data = transform(data)
    models = [m for m in args["models"].split(",")]
    eva = Evaluation()

    eva.register_task("meanspeed", init_meanspeed(args, network, seed))

    # adj = np.loadtxt(
    #     "../models/training/gtn_precalc_adj/traj_adj_k_2.gz"
    # )  # change to desired path
    for m in models:
        model, margs, model_path = model_map[m]
        if "con" in m or "add" in m:  # case where its an aggregtaed embedding
            agg_models = []
            for agg_model_name in margs["models"]:
                agg_model, agg_margs, agg_model_path = model_map[agg_model_name]
                # agg_margs["adj"] = adj_bi
                # agg_margs["network"] = network
                # print(agg_margs, data.x.shape)
                # agg_model = agg_model(data, device=device, network=network, adj=adj_bi)
                # print(get_model_path_for_task(agg_model_path, ["meanspeed"], agg_model_name))
                # agg_model.load_model(
                #     path=get_model_path_for_task(agg_model_path, ["meanspeed"], agg_model_name)
                # )
                # gtc = GTCModel(data, device, network, adj=adj_bi)
                # gtc.load_model("../model_states/gtc/model_base.pt")
                # t2v = Traj2VecModel(data, network, adj_bi, device=device)
                # t2v.load_model("../model_states/traj2vec/model_base.pt")
                #print(agg_model.load_emb())
                #agg_models.append(agg_model)
                # agg_models.append(gtc)
                # agg_models.append(t2v)
                break

            margs["models"] = agg_models

        #model = ConcateAdapterModel(None, None, models=[gtc, t2v], aggregator="concate")

        gtc_test = GTCModel(data, device, network, adj=adj_bi)
        gtc_test.load_model("../model_states/gtc/model_base.pt")
        t2v_test = Traj2VecModel(data, network, adj_bi, device=device)
        t2v_test.load_model("../model_states/traj2vec/model_base.pt")
        model_r = model(data, device=device, models=[gtc, t2v], aggregator="concate")
        model_test = model(data, device=device, models=[gtc_test, t2v_test], aggregator="concate")
        print(gtc_test.load_emb())
        print((gtc_test.load_emb()==gtc.load_emb()).all())
        print((t2v_test.load_emb()==t2v.load_emb()).all())
        print(torch.equal(gtc_test.train_data.edge_traj_index, gtc.train_data.edge_traj_index))
        eva.register_model(m, model_r)
        eva.register_model("test", model_test)

    #res = eva.run()

    # return res

In [17]:
res = evaluate_model({"models": "con-traj2vec-gtc"}, data_rest, network, test, 88)
print(res)

[[-0.03461973 -0.7446656  -0.36518294 ... -0.12940633 -0.19541745
  -0.41113168]
 [-0.06902651 -0.30286184 -0.42417783 ... -0.11377949 -0.24691841
  -0.15814777]
 [-0.10037328 -0.67797256 -0.45901316 ... -0.14502342 -0.143866
  -0.5151305 ]
 ...
 [-0.03662838  0.23067358  0.02284841 ...  0.09844947 -0.02313899
   0.28437442]
 [-0.52729726 -0.24934813  0.35677475 ... -0.5402291  -0.09656639
   0.44079298]
 [-0.52729726 -0.24934813  0.35677475 ... -0.5402291  -0.09656639
   0.44079298]]
False
True
True
None


In [21]:
gtc_test = GTCModel(data_rest, device, network, adj=adj_bi)
gtc_test.load_model("../model_states/gtc/model_base.pt")

gtc_test.load_emb()

array([[-0.03461973, -0.7446656 , -0.36518294, ..., -0.12940633,
        -0.19541745, -0.41113168],
       [-0.06902651, -0.30286184, -0.42417783, ..., -0.11377949,
        -0.24691841, -0.15814777],
       [-0.10037328, -0.67797256, -0.45901316, ..., -0.14502342,
        -0.143866  , -0.5151305 ],
       ...,
       [-0.03662838,  0.23067358,  0.02284841, ...,  0.09844947,
        -0.02313899,  0.28437442],
       [-0.52729726, -0.24934813,  0.35677475, ..., -0.5402291 ,
        -0.09656639,  0.44079298],
       [-0.52729726, -0.24934813,  0.35677475, ..., -0.5402291 ,
        -0.09656639,  0.44079298]], dtype=float32)

In [80]:
travel_time_est = TravelTimeEstimation(
    traj_dataset=trajectory,
    network=network,
    device=device,
    batch_size=128,
    epochs=3,
    seed=88,
)
travel_time_est.register_metric(
    name="MSE", metric_func=metrics.mean_squared_error, args={}
)
travel_time_est.register_metric(
    name="MAE", metric_func=metrics.mean_absolute_error, args={}
)

# for i, (m, e) in enumerate(models):
# m.train(epochs=e)
# zn = m.load_emb()
# zcn = np.concatenate((zn, z2), axis=1)
# zct = np.concatenate((zn, z3), axis=1)
# X = z # embedding for each node
eva = [z, z2, gtc.load_emb(), gtc2.load_emb(), gae_emb, rand_emb]
for X in eva:
    print(travel_time_est.evaluate(X))

{'MSE': 9949.570608700878, 'MAE': 71.89226948661805}
{'MSE': 11646.49069475799, 'MAE': 81.10363915457725}
{'MSE': 10092.045623252046, 'MAE': 72.39778632850647}


In [17]:
nextlocation_pred = NextLocationPrediciton(
    traj_dataset=test,
    network=network,
    device=device,
    batch_size=256,
    epochs=5,
    seed=88,
)

nextlocation_pred.register_metric(
    name="accuracy",
    metric_func=metrics.accuracy_score,
    args={"normalize": True},
)

# for i, (m, e) in enumerate(models):
#     m.train(epochs=e)
#     zn = m.load_emb()
#     zcn = np.concatenate((zn, z2), axis=1)
#     zct = np.concatenate((zn, z3), axis=1)
#     zcnn = np.concatenate((zn, z4), axis=1)
#     zctn = np.concatenate((zn, z5), axis=1)
    # X = z # embedding for each node
eva = [z, z2, gtc.load_emb(), gtc2.load_emb(), gae_emb, rand_emb]
for X in eva:
    print(nextlocation_pred.evaluate(X))

Pandas Apply:   0%|          | 0/370616 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/370616 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/92655 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/92655 [00:00<?, ?it/s]

Average training loss in episode 0: 78.67120297574206
Average training loss in episode 1: 36.788649095356135


KeyboardInterrupt: 